## Required imports

In [1]:
from collections import defaultdict

from pprint import pprint
import nltk
import numpy as np
import tensorflow as tf
import re
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import pickle
from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import plot_model
from keras.utils.np_utils import to_categorical
import pandas as pd
import cleantext
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers import Dense, Dropout
# from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import confusion_matrix
# from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords

## GPU check

In [2]:
print(len(tf.config.experimental.list_physical_devices('GPU')))

1


In [3]:
print("Num GPUs Available: ", (tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Reading datasets from file

In [4]:
topical_chat  = pd.read_csv('dataset/topical_chat.csv')
topical_chat.sentiment.unique()

array([' Curious to dive deeper', ' Happy', ' Neutral', ' Surprised',
       ' Disgusted', ' Sad', ' Fearful', ' Angry'], dtype=object)

In [5]:
emotion_emotion = pd.read_csv('dataset/emotion-emotion_69k.csv')
emotion_emotion.emotion.unique()

array(['sentimental', 'afraid', 'proud', 'faithful', 'terrified',
       'joyful', 'angry', 'sad', 'jealous', 'grateful', 'prepared',
       'embarrassed', 'excited', 'annoyed', 'lonely', 'ashamed', 'guilty',
       'surprised', 'nostalgic', 'confident', 'furious', 'disappointed',
       'caring', 'trusting', 'disgusted', 'anticipating', 'anxious',
       'hopeful', 'content', 'impressed', 'apprehensive', 'devastated',
       ' I really killed it!', ' we were in a different country',
       't even like scary things',
       't believe I like the show Power so much. I was never really into shows like that',
       nan, ' time to jump on the motorcycle and go cruising!',
       " a boy.  I hear all these different labor stories that aren't exactly reassuring!  ",
       't believe my daughter taught herself how to play the ukelele. I was amazed',
       't think I wold like super heroes',
       "m so mad with my brother. He stole from me and didn't think I would notice. ",
       " but

In [6]:
topical_chat.head()

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper


In [7]:
emotion_emotion.head()
# unique_emotions = list()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...,NaN,NaN
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.,NaN,NaN
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.,NaN,NaN


In [8]:
# sentimental, caring,
# afraid, terrified
# proud, impressed
# faithful, trusting, confident ,grateful,hopeful
# joyful, happy
# angry, furious
# sad, devastated, disappointed, lonely
# annoyed, ashamed, embarrassed, guilty
# disgusted, jealous
# anticipating, anxious, apprehensive,
# neutral, content, prepared ,nostalgic
# surprised, curious, excited

## Preprocessing and appending datasets

In [9]:
emotion_subset = emotion_emotion[['Situation', 'emotion']]
emotion_subset.rename(columns = {'Situation':'Line', 'emotion':'Sentiment'}, inplace = True)

topical_subset = topical_chat[['message', 'sentiment']]
topical_subset.rename(columns = {'message':'Line', 'sentiment':'Sentiment'}, inplace = True)

emotion_subset.drop_duplicates(subset='Line', keep='first', inplace=True)
emotion_subset.head()

c:\users\shrir\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
c:\users\shrir\appdata\local\programs\python\python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Line,Sentiment
0,I remember going to the fireworks with my best...,sentimental
5,i used to scare for darkness,afraid
10,I showed a guy how to run a good bead in weldi...,proud
14,I have always been loyal to my wife.,faithful
17,A recent job interview that I had made me feel...,terrified


In [10]:
emotion_emotion.emotion.value_counts()

surprised                                                                                                                                                                                         3295
excited                                                                                                                                                                                           2465
angry                                                                                                                                                                                             2296
proud                                                                                                                                                                                             2247
annoyed                                                                                                                                                                                           2213
sad  

In [11]:
topical_chat.sentiment.value_counts()

 Curious to dive deeper    80888
 Neutral                   41367
 Surprised                 30638
 Happy                     29617
 Sad                        2533
 Disgusted                  1433
 Fearful                    1026
 Angry                       876
Name: sentiment, dtype: int64

In [12]:
topical_subset.head()
print(topical_subset.shape)


(188378, 2)


In [19]:
# rus  = RandomUnderSampler(random_state=69)
# topical_subset_us, _ = rus.fit_resample(topical_subset, topical_subset['Sentiment'])
# topical_subset_us = pd.DataFrame(topical_subset_us, columns=['Line','Sentiment'])
#
# topical_subset_us.reset_index(inplace = True, drop = True)
#

In [20]:
# topical_subset_us.head()
#

,Line,Sentiment
0,Totally agree. It's ridiculous that we don't ...,Angry
1,All they mentioned was that it was a truly he...,Angry
2,Right there is not one source out there that ...,Angry
3,I think so that they can attempt to make them...,Angry
4,Agreed! It's almost as ignorant as the judge ...,Angry


In [21]:
# topical_subset_us.Sentiment.value_counts()

 Angry                     876
 Curious to dive deeper    876
 Disgusted                 876
 Fearful                   876
 Happy                     876
 Neutral                   876
 Sad                       876
 Surprised                 876
Name: Sentiment, dtype: int64

In [72]:
df_new = emotion_subset
# df_new = df_new.append(topical_subset_us, ignore_index=True)
df_new = df_new.append(topical_subset, ignore_index=True)
df_new.tail()


,Line,Sentiment
207578,"Wow, it does not seem like that long. Since I...",Surprised
207579,"I havent seen that episode, I might google it...",Curious to dive deeper
207580,I don't think I have either. That's an insane...,Curious to dive deeper
207581,"I did, my little brother used to love Thomas ...",Happy
207582,"It did. Ringo Starr, George Carlin, and Alec ...",Neutral


In [73]:
print(df_new.Sentiment.unique())

['sentimental' 'afraid' 'proud' 'faithful' 'terrified' 'joyful' 'angry'
 'sad' 'jealous' 'grateful' 'prepared' 'embarrassed' 'excited' 'annoyed'
 'lonely' 'ashamed' 'guilty' 'surprised' 'nostalgic' 'confident' 'furious'
 'disappointed' 'caring' 'trusting' 'disgusted' 'anticipating' 'anxious'
 'hopeful' 'content' 'impressed' 'apprehensive' 'devastated'
 ' I really killed it!' ' we were in a different country'
 't even like scary things'
 't believe I like the show Power so much. I was never really into shows like that'
 nan ' time to jump on the motorcycle and go cruising!'
 " a boy.  I hear all these different labor stories that aren't exactly reassuring!  "
 't think I wold like super heroes'
 "m so mad with my brother. He stole from me and didn't think I would notice. "
 " but what I didn't know was that he was working in the next room with the door open.  He approached and asked what I had been saying.  I knew I was caught.  I was so disgusted with myself.  "
 '(' ' Curious to dive 

In [74]:
unique_sentiments = ['sentimental', 'afraid', 'proud', 'faithful', 'joyful', 'angry', 'sad', 'annoyed', 'disgusted', 'anticipating', 'neutral', 'surprised']

In [75]:
for i in range(len(df_new)):
    senti = str(df_new.Sentiment[i])
    if senti == 'caring':
        df_new.Sentiment[i] = 'sentimental'
    elif senti == ' Happy':
        df_new.Sentiment[i] = 'joyful'
    elif senti == 'terrified' or senti == ' Fearful':
        df_new.Sentiment[i] = 'afraid'
    elif senti == 'impressed':
        df_new.Sentiment[i] = 'proud'
    elif senti == 'trusting' or senti == 'confident' or senti == 'grateful' or senti == 'hopeful':
        df_new.Sentiment[i] = 'faithful'
    elif senti == 'furious' or senti == ' Angry':
        df_new.Sentiment[i] = 'angry'
    elif senti == 'devastated' or senti == 'disappointed' or senti == 'lonely' or senti == ' Sad':
        df_new.Sentiment[i] = 'sad'
    elif senti == 'ashamed' or senti == 'embarrassed' or senti == 'guilty':
        df_new.Sentiment[i] = 'annoyed'
    if senti == 'jealous' or senti == ' Disgusted':
        df_new.Sentiment[i] = 'disgusted'
    elif senti == 'anxious' or senti == 'apprehensive':
        df_new.Sentiment[i] = 'anticipating'
    elif senti == 'content' or senti == 'prepared' or senti == 'nostalgic' or senti == ' Neutral':
        df_new.Sentiment[i] = 'neutral'
    elif senti == ' Curious to dive deeper' or senti == 'excited' or senti == ' Surprised':
        df_new.Sentiment[i] = 'surprised'

df_new.Sentiment.unique()

array(['sentimental', 'afraid', 'proud', 'faithful', 'joyful', 'angry',
       'sad', 'disgusted', 'neutral', 'annoyed', 'surprised',
       'anticipating', ' I really killed it!',
       ' we were in a different country', 't even like scary things',
       't believe I like the show Power so much. I was never really into shows like that',
       nan, ' time to jump on the motorcycle and go cruising!',
       " a boy.  I hear all these different labor stories that aren't exactly reassuring!  ",
       't think I wold like super heroes',
       "m so mad with my brother. He stole from me and didn't think I would notice. ",
       " but what I didn't know was that he was working in the next room with the door open.  He approached and asked what I had been saying.  I knew I was caught.  I was so disgusted with myself.  ",
       '('], dtype=object)

In [76]:
# drop rows from above list
df_new = df_new[df_new.Sentiment.isin(unique_sentiments) == True]
df_new.Sentiment.unique()


array(['sentimental', 'afraid', 'proud', 'faithful', 'joyful', 'angry',
       'sad', 'disgusted', 'neutral', 'annoyed', 'surprised',
       'anticipating'], dtype=object)

In [77]:
sentiment_idx_dict = defaultdict()
for i in range(len(unique_sentiments)):
    sentiment_idx_dict[unique_sentiments[i]] = i

In [78]:
line  = list()
sentiment_list = list()


In [83]:
df_new.reset_index(inplace = True, drop = True)
df_new.head()

,Line,Sentiment
0,I remember going to the fireworks with my best...,sentimental
1,i used to scare for darkness,afraid
2,I showed a guy how to run a good bead in weldi...,proud
3,I have always been loyal to my wife.,faithful
4,A recent job interview that I had made me feel...,afraid


In [84]:
for i in range(len(df_new)):
    # print(i)
    line.append(str(df_new['Line'][i]).strip())
    sentiment_list.append(sentiment_idx_dict[str(df_new['Sentiment'][i])])

In [85]:
print(len(line) , len(sentiment_list))

415144 415144


In [86]:
print(unique_sentiments)


['sentimental', 'afraid', 'proud', 'faithful', 'joyful', 'angry', 'sad', 'annoyed', 'disgusted', 'anticipating', 'neutral', 'surprised']


In [87]:
def contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"wouldn't", "would not",s)
    s = re.sub(r"couldn't", "could not",s)
    s = re.sub(r"didn't", "did not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
# data[‘pre_process']=data[‘pre_process'].apply(lambda x:contractions(x))

def lemmatize(ip):
    lemmatizer = WordNetLemmatizer()
    lemmatized_lines = []
    for line in ip:
        new_line = ''
        tokens = nltk.word_tokenize(line)
        for t in tokens:
            t = contractions(t)
            lemmatizer.lemmatize(t)
            new_line += t + ' '
        new_line.strip()
        lemmatized_lines.append(new_line)
    return lemmatized_lines


def clean(ip):
    clean_lines = list()
    for i , sentences in enumerate(ip):
        # print(ip[i])
        clean_line = (cleantext.clean(str(ip[i]), extra_spaces=True, lowercase=True, stopwords=True, stemming=False, numbers=True, punct=True, clean_all = True))
        clean_lines.append(clean_line)
    return clean_lines


## Cleaning dataset

In [88]:
df_sentiment = pd.DataFrame(list(zip(line, sentiment_list)), columns=['Line', 'Emotion'])
print(df_sentiment.head())

                                                Line  Emotion
0  I remember going to the fireworks with my best...        0
1                       i used to scare for darkness        1
2  I showed a guy how to run a good bead in weldi...        2
3               I have always been loyal to my wife.        3
4  A recent job interview that I had made me feel...        1


In [89]:
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: contractions(x))
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: ' '.join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: re.sub(' +', ' ', x))

stop = stopwords.words('english')
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: ' '.join([x for x in x.split() if x not in stop]))
lemmatizer = WordNetLemmatizer()
df_sentiment['Line'] = df_sentiment['Line'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))


In [90]:
df_sentiment.head()

,Line,Emotion
0,remember going firework best friend lot people...,0
1,used scare darkness,1
2,showed guy run good bead welding class caught ...,2
3,always loyal wife,3
4,recent job interview made feel anxious felt li...,1


In [91]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415144 entries, 0 to 415143
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Line     415144 non-null  object
 1   Emotion  415144 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.3+ MB


In [93]:
df_sentiment = df_sentiment[df_sentiment['Line'].notna()]

In [94]:
df_sentiment['Emotion'].value_counts()

11    226500
10     86228
4      60426
6       9940
3       5460
8       5246
7       4620
1       4514
5       4290
9       3334
2       2554
0       2032
Name: Emotion, dtype: int64

In [95]:
df_sentiment.to_csv('dataset/line_sentiment.csv', index=False)


## Reading saved dataset

In [4]:
df_sentiment = pd.read_csv('dataset/line_sentiment.csv')
df_sentiment.head()

,Line,Emotion
0,remember going firework best friend lot people...,0
1,used scare darkness,1
2,showed guy run good bead welding class caught ...,2
3,always loyal wife,3
4,recent job interview made feel anxious felt li...,1


## Vectorizing inputs

In [16]:
df_sentiment = df_sentiment[df_sentiment['Line'].notna()]
X_train, X_test, Y_train, Y_test = train_test_split(df_sentiment['Line'], df_sentiment['Emotion'], random_state=42, train_size=0.9)
print('Train: ',X_train.shape,Y_train.shape,'Test: ',(X_test.shape,Y_test.shape))

Train:  (372301,) (372301,) Test:  ((41367,), (41367,))


In [19]:
print(X_train)

129991    hey know netflix make percent internet bandwid...
161547                            say human side effect hum
204452    saw president indonesia released three pop alb...
128660    like idea crime fighting duo using subway woul...
238782    lol right kind miss though like going store pi...
                                ...                        
260051    really know almost hobbit character name taken...
367121    yeah feed addiction clicking fb mostly blue zu...
132382                            yes use play nintendo kid
147366                  surprised lasted long display color
122376                         sued million even make sense
Name: Line, Length: 372301, dtype: object


In [6]:
print('TFIDF Vectorizer……')
vectorizer = TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


In [27]:
print(type(X_test))

<class 'pandas.core.series.Series'>


In [7]:
print(tf_x_test)


  (0, 34695)	0.49169857843455916
  (0, 32466)	0.245651782509468
  (0, 25138)	0.3908650135451507
  (0, 21715)	0.14645538503268263
  (0, 19648)	0.39128059852921226
  (0, 17997)	0.4635974844806341
  (0, 17786)	0.24855348760564516
  (0, 17572)	0.250867727159096
  (0, 10453)	0.17585927318052288
  (1, 30222)	0.32170127940111987
  (1, 26564)	0.28814353759393396
  (1, 23387)	0.20337748770115177
  (1, 20364)	0.2093034795564782
  (1, 20177)	0.18878858684785316
  (1, 16576)	0.24080541572755182
  (1, 15261)	0.5593425626936508
  (1, 13197)	0.3788415706724859
  (1, 8078)	0.38147123294352775
  (1, 995)	0.18104563257635967
  (2, 35003)	0.2715127362499651
  (2, 29453)	0.4502987125082147
  (2, 24084)	0.3533564370968893
  (2, 23685)	0.4765782329115876
  (2, 15492)	0.5117043769274519
  (2, 13451)	0.3311841639901435
  :	:
  (41364, 17143)	0.1919310720564133
  (41364, 13061)	0.3036236351772873
  (41364, 5122)	0.32619299075530306
  (41364, 1469)	0.4132284796458545
  (41365, 34781)	0.14437420152375438
  (4136

## Logistic Regression models

### liblinear solver

In [19]:
lr_ll = LogisticRegression(max_iter=1000,solver='liblinear')
lr_ll.fit(tf_x_train,Y_train)
y_test_pred = lr_ll.predict(tf_x_test)
report_lr_ll = classification_report(Y_test, y_test_pred,output_dict=True)

In [20]:
pprint(report_lr_ll)
print(report_lr_ll['accuracy'])

{'0': {'f1-score': 0.39024390243902435,
       'precision': 0.5773195876288659,
       'recall': 0.29473684210526313,
       'support': 190},
 '1': {'f1-score': 0.4654654654654655,
       'precision': 0.7635467980295566,
       'recall': 0.3347732181425486,
       'support': 463},
 '10': {'f1-score': 0.3760454002389486,
        'precision': 0.5370014928556195,
        'recall': 0.28932551993565436,
        'support': 8703},
 '11': {'f1-score': 0.7545730605162316,
        'precision': 0.6361586053457682,
        'recall': 0.9271532003731178,
        'support': 22513},
 '2': {'f1-score': 0.48793565683646123,
       'precision': 0.7777777777777778,
       'recall': 0.35546875,
       'support': 256},
 '3': {'f1-score': 0.5324074074074074,
       'precision': 0.6515580736543909,
       'recall': 0.4500978473581213,
       'support': 511},
 '4': {'f1-score': 0.22484439834024894,
       'precision': 0.5139300533491404,
       'recall': 0.14390041493775935,
       'support': 6025},
 '5': {'f1

### newton-cg solver

In [21]:
lr_ncg = LogisticRegression(max_iter=1000,solver='newton-cg')
lr_ncg.fit(tf_x_train,Y_train)
y_test_pred = lr_ncg.predict(tf_x_test)
report_lr_ncg = classification_report(Y_test, y_test_pred,output_dict=True)

In [22]:
pprint(report_lr_ncg)
print(report_lr_ncg['accuracy'])

{'0': {'f1-score': 0.42675159235668786,
       'precision': 0.5403225806451613,
       'recall': 0.3526315789473684,
       'support': 190},
 '1': {'f1-score': 0.5102639296187683,
       'precision': 0.7945205479452054,
       'recall': 0.3758099352051836,
       'support': 463},
 '10': {'f1-score': 0.400262754543464,
        'precision': 0.5486194477791116,
        'recall': 0.31506377111340916,
        'support': 8703},
 '11': {'f1-score': 0.7580277757366449,
        'precision': 0.6463051718196912,
        'recall': 0.9164482743303869,
        'support': 22513},
 '2': {'f1-score': 0.5333333333333333,
       'precision': 0.7248322147651006,
       'recall': 0.421875,
       'support': 256},
 '3': {'f1-score': 0.5555555555555556,
       'precision': 0.611764705882353,
       'recall': 0.5088062622309197,
       'support': 511},
 '4': {'f1-score': 0.2528358961431813,
       'precision': 0.5254059717129387,
       'recall': 0.16647302904564315,
       'support': 6025},
 '5': {'f1-score'

### lbfgs solver

In [12]:
lr_lbfgs = LogisticRegression(max_iter=1000,solver='lbfgs')
lr_lbfgs.fit(tf_x_train,Y_train)
y_test_pred = lr_lbfgs.predict(tf_x_test)
report_lr_lbfgs = classification_report(Y_test, y_test_pred,output_dict=True)

In [13]:
pprint(report_lr_lbfgs)
print(report_lr_lbfgs['accuracy'])

{'0': {'f1-score': 0.42675159235668786,
       'precision': 0.5403225806451613,
       'recall': 0.3526315789473684,
       'support': 190},
 '1': {'f1-score': 0.5102639296187683,
       'precision': 0.7945205479452054,
       'recall': 0.3758099352051836,
       'support': 463},
 '10': {'f1-score': 0.4003796451777762,
        'precision': 0.5490588706447738,
        'recall': 0.31506377111340916,
        'support': 8703},
 '11': {'f1-score': 0.7580505905909584,
        'precision': 0.6463162510963538,
        'recall': 0.9164926931106472,
        'support': 22513},
 '2': {'f1-score': 0.5333333333333333,
       'precision': 0.7248322147651006,
       'recall': 0.421875,
       'support': 256},
 '3': {'f1-score': 0.5555555555555556,
       'precision': 0.611764705882353,
       'recall': 0.5088062622309197,
       'support': 511},
 '4': {'f1-score': 0.2528040327662256,
       'precision': 0.525130890052356,
       'recall': 0.16647302904564315,
       'support': 6025},
 '5': {'f1-score'

### sag solver

In [14]:
lr_sag = LogisticRegression(max_iter=1000,solver='sag')
lr_sag.fit(tf_x_train,Y_train)
y_test_pred = lr_sag.predict(tf_x_test)
report_lr_sag = classification_report(Y_test, y_test_pred,output_dict=True)

In [15]:
pprint(report_lr_sag)
print(report_lr_sag['accuracy'])


{'0': {'f1-score': 0.42675159235668786,
       'precision': 0.5403225806451613,
       'recall': 0.3526315789473684,
       'support': 190},
 '1': {'f1-score': 0.5102639296187683,
       'precision': 0.7945205479452054,
       'recall': 0.3758099352051836,
       'support': 463},
 '10': {'f1-score': 0.4002919708029198,
        'precision': 0.5487292375425256,
        'recall': 0.31506377111340916,
        'support': 8703},
 '11': {'f1-score': 0.7580505905909584,
        'precision': 0.6463162510963538,
        'recall': 0.9164926931106472,
        'support': 22513},
 '2': {'f1-score': 0.5333333333333333,
       'precision': 0.7248322147651006,
       'recall': 0.421875,
       'support': 256},
 '3': {'f1-score': 0.5555555555555556,
       'precision': 0.611764705882353,
       'recall': 0.5088062622309197,
       'support': 511},
 '4': {'f1-score': 0.2528358961431813,
       'precision': 0.5254059717129387,
       'recall': 0.16647302904564315,
       'support': 6025},
 '5': {'f1-score

### saga solver

In [16]:
lr_saga = LogisticRegression(max_iter=1000,solver='saga')
lr_saga.fit(tf_x_train,Y_train)
y_test_pred = lr_saga.predict(tf_x_test)
report_lr_saga = classification_report(Y_test, y_test_pred,output_dict=True)

In [17]:
pprint(report_lr_saga)
print(report_lr_saga['accuracy'])

{'0': {'f1-score': 0.42675159235668786,
       'precision': 0.5403225806451613,
       'recall': 0.3526315789473684,
       'support': 190},
 '1': {'f1-score': 0.5102639296187683,
       'precision': 0.7945205479452054,
       'recall': 0.3758099352051836,
       'support': 463},
 '10': {'f1-score': 0.40032119132783417,
        'precision': 0.5488390712570056,
        'recall': 0.31506377111340916,
        'support': 8703},
 '11': {'f1-score': 0.7580277757366449,
        'precision': 0.6463051718196912,
        'recall': 0.9164482743303869,
        'support': 22513},
 '2': {'f1-score': 0.5333333333333333,
       'precision': 0.7248322147651006,
       'recall': 0.421875,
       'support': 256},
 '3': {'f1-score': 0.5555555555555556,
       'precision': 0.611764705882353,
       'recall': 0.5088062622309197,
       'support': 511},
 '4': {'f1-score': 0.2528358961431813,
       'precision': 0.5254059717129387,
       'recall': 0.16647302904564315,
       'support': 6025},
 '5': {'f1-scor

### saving logistic regression models

In [23]:
filename = 'lr_ll.sav'
pickle.dump(lr_ll, open(f'models/{filename}', 'wb'))

filename = 'lr_ncg.sav'
pickle.dump(lr_ncg, open(f'models/{filename}', 'wb'))

filename = 'lr_lbfgs.sav'
pickle.dump(lr_lbfgs, open(f'models/{filename}', 'wb'))

filename = 'lr_sag.sav'
pickle.dump(lr_sag, open(f'models/{filename}', 'wb'))

filename = 'lr_saga.sav'
pickle.dump(lr_saga, open(f'models/{filename}', 'wb'))


### Loading saved linear regression models

In [ ]:
filename = 'lr_ll.sav'
lr_ll = pickle.load(open(f'models/{filename}', 'rb'))

filename = 'lr_ncg.sav'
lr_ncg = pickle.load(open(f'models/{filename}', 'rb'))

filename = 'lr_lbfgs.sav'
lr_lbfgs = pickle.load(open(f'models/{filename}', 'rb'))

filename = 'lr_sag.sav'
lr_sag = pickle.load(open(f'models/{filename}', 'rb'))

filename = 'lr_saga.sav'
lr_saga = pickle.load(open(f'models/{filename}', 'rb'))


## SVM Classifier

In [24]:
svm = LinearSVC(random_state=0)
svm.fit(tf_x_train,Y_train)
y_test_pred = svm.predict(tf_x_test)
report_svm = classification_report(Y_test, y_test_pred,output_dict=True)

In [25]:
pprint(report_svm)
print(report_svm['accuracy'])

{'0': {'f1-score': 0.6301369863013698,
       'precision': 0.6571428571428571,
       'recall': 0.6052631578947368,
       'support': 190},
 '1': {'f1-score': 0.6287978863936593,
       'precision': 0.8095238095238095,
       'recall': 0.5140388768898488,
       'support': 463},
 '10': {'f1-score': 0.4352872896111434,
        'precision': 0.5904349537492619,
        'recall': 0.3447087211306446,
        'support': 8703},
 '11': {'f1-score': 0.7736338028169013,
        'precision': 0.670136968474477,
        'recall': 0.9149380358015369,
        'support': 22513},
 '2': {'f1-score': 0.7004219409282699,
       'precision': 0.7614678899082569,
       'recall': 0.6484375,
       'support': 256},
 '3': {'f1-score': 0.6872509960159363,
       'precision': 0.6997971602434077,
       'recall': 0.675146771037182,
       'support': 511},
 '4': {'f1-score': 0.33949739212897106,
       'precision': 0.593944421401908,
       'recall': 0.23767634854771785,
       'support': 6025},
 '5': {'f1-score':

### saving svm models

In [26]:
filename = 'svm.sav'
pickle.dump(lr_ll, open(f'models/{filename}', 'wb'))


### Loading saved svm model

In [ ]:
filename = 'svm.sav'
svm = pickle.load(open(f'models/{filename}', 'rb'))


## SGD model

In [28]:
sgd = SGDClassifier()
sgd.fit(tf_x_train, Y_train)
y_test_pred = sgd.predict(tf_x_test)
report_sgd = classification_report(Y_test, y_test_pred,output_dict=True)


In [29]:
pprint(report_sgd)
print(report_sgd['accuracy'])


{'0': {'f1-score': 0.3296703296703297,
       'precision': 0.5421686746987951,
       'recall': 0.23684210526315788,
       'support': 190},
 '1': {'f1-score': 0.35725677830940994,
       'precision': 0.6829268292682927,
       'recall': 0.24190064794816415,
       'support': 463},
 '10': {'f1-score': 0.11409604812778756,
        'precision': 0.5863539445628998,
        'recall': 0.06319659887395151,
        'support': 8703},
 '11': {'f1-score': 0.732438545226421,
        'precision': 0.5828428796636889,
        'recall': 0.985341802514103,
        'support': 22513},
 '2': {'f1-score': 0.3856041131105399,
       'precision': 0.5639097744360902,
       'recall': 0.29296875,
       'support': 256},
 '3': {'f1-score': 0.42821158690176325,
       'precision': 0.6007067137809188,
       'recall': 0.33268101761252444,
       'support': 511},
 '4': {'f1-score': 0.09627282329862684,
       'precision': 0.5299003322259136,
       'recall': 0.05294605809128631,
       'support': 6025},
 '5': {'f

### saving sgd models

In [30]:
filename = 'sgd.sav'
pickle.dump(lr_ll, open(f'models/{filename}', 'wb'))

### Loading saved svm model

In [ ]:
filename = 'sgd.sav'
sgd = pickle.load(open(f'models/{filename}', 'rb'))


## DNN model

### preprocessing

In [5]:
df_sentiment = df_sentiment[df_sentiment['Line'].notna()]
y = df_sentiment.iloc[:, 1].values
y = np.array([int(i) for i in y])
print(y)

[ 0  1  2 ... 11  4 10]


In [14]:
# split data into training and validation set in ratio of 9:1
X_train, X_test, Y_train, Y_test = train_test_split(df_sentiment['Line'], y, random_state=42, train_size=0.9, stratify=y)

print('No. of samples Training data :', X_train.shape[0])
print('No. of samples Validation data :', X_test.shape[0])

No. of samples Training data : 372301
No. of samples Validation data : 41367


In [15]:
with open('processed_dataset/'+"sentiment_classifer" +'.pickle','wb') as f:
        pickle.dump([X_train],f)

In [7]:
print ("TF-IDF on text data ... ")
tfidf = TfidfVectorizer(binary=True)
X_train = tfidf.fit_transform(X_train).astype('float16')
X_test = tfidf.transform(X_test).astype('float16')
# print(f'train_text[0]: {X_train[0]}')
# print(f'X[0]: {X[0]}')
# print(f'X shape: {X.shape}')

ip_dim = X_train.shape[1]

TF-IDF on text data ... 


In [21]:
f = open("tfdf.pickle", "wb+")
pickle.dump(tfidf, f)
f.close()

In [8]:
print(Y_train[0])


11


In [9]:
num_classes = np.max(Y_train) + 1
Y_train_onehot = to_categorical(Y_train, num_classes=num_classes)
Y_test_onehot = to_categorical(Y_test, num_classes=num_classes)
print(num_classes)



12


In [11]:
# Model Training
print ("Create model ... ")
def build_model():
    model = Sequential()
    model.add(Dense(256, input_dim=ip_dim, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(160, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision()])
    return model


Create model ... 


### Training

In [12]:
print("Compile model ...")
model_dnn = build_model()
model_dnn.fit(X_train, Y_train_onehot, epochs=15, batch_size=128, validation_data=(X_test, Y_test_onehot))

Compile model ...
Epoch 1/15


c:\Users\ACER\anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


2909/2909 [==============================] - 45s 14ms/step - loss: 1.1510 - accuracy: 0.5781 - precision: 0.6767 - val_loss: 1.0040 - val_accuracy: 0.6236 - val_precision: 0.7140
Epoch 2/15
2909/2909 [==============================] - 42s 14ms/step - loss: 0.9066 - accuracy: 0.6735 - precision: 0.7440 - val_loss: 0.8384 - val_accuracy: 0.6971 - val_precision: 0.7704
Epoch 3/15
2909/2909 [==============================] - 43s 15ms/step - loss: 0.6930 - accuracy: 0.7571 - precision: 0.8110 - val_loss: 0.7194 - val_accuracy: 0.7456 - val_precision: 0.7909
Epoch 4/15
2909/2909 [==============================] - 46s 16ms/step - loss: 0.5539 - accuracy: 0.8079 - precision: 0.8524 - val_loss: 0.6145 - val_accuracy: 0.7874 - val_precision: 0.8328
Epoch 5/15
2909/2909 [==============================] - 45s 15ms/step - loss: 0.4663 - accuracy: 0.8395 - precision: 0.8763 - val_loss: 0.5528 - val_accuracy: 0.8147 - val_precision: 0.8555
Epoch 6/15
2909/2909 [==============================] - 46s 1

### Saving dnn model

In [13]:
filename = 'model_dnn.h5'
model_dnn.save(f'models/{filename}')

### Loading saved dnn model

In [ ]:
filename = 'model_dnn.h5'
model_dnn = load_model(f'models/{filename}')